In [2]:
%load_ext autoreload
%autoreload 2

import spglib as spg
import ase
from ase.visualize import view
# the FIRE optimizer seems much better than the BFGS
from ase.optimize import BFGS, FIRE
from ase.calculators.lj import LennardJones
import ase.spacegroup as aspg
import numpy as np
import pandas as pd
from collections import Iterable
import pickle as pck

import sys, os
# add to PYTHONPATH
modulepath = '/home/musil/Dropbox/Felix_work/qmat/spglib-test/'
#modulepath = '/Users/iMac/Dropbox/Felix_work/qmat/spglib-test/'
if modulepath not in sys.path:
    sys.path.insert(0,modulepath)

from genGeneralSites import GenCrystalStructures

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ImportError: No module named genGeneralSites

In [6]:
from genGeneralSites import GenCrystalStructures

## Functions to print blocks of QE inputs

In [7]:
def makeCard(cardDict,cardKeys=None,optionKey='unit',T=False, endl = ' \n',eq = ' = ',stl = '  '):
    """Transform a Card block in dict form into a string."""
    from itertools import izip_longest
    from  tabulate import tabulate
    cardStr = ''
    if cardKeys is None:
        keys =  cardDict.keys()
        keys.remove(optionKey)
        cardKeys = cardDict[keys[0]]
        
    if optionKey in cardDict.keys():
        keys =  cardDict.keys()
        keys.remove(optionKey)
        for key in keys:
            cardStr += key + stl
        cardStr += cardDict[optionKey] + endl

        for key,val in cardDict.items():
            if isinstance(val, dict):
                l = [val[cardKey] for cardKey in cardKeys] 
                ll = map(list, izip_longest(*l)) if T else l
                cardStr += stl + tabulate(ll, tablefmt="plain").replace('[','').replace(']','').replace(',','').replace('\n',endl+stl) + endl
        
        return cardStr
    else:
        print 'cardDict has no key called: {} (optionKey)'.format(optionKey)
def makeNamelist(namelistName,namelistDict,namelistKeys=None, endl = ' \n',eq = ' = ',stl = '  '):
    """Transform a Name List block in dict form into a string."""
    if namelistKeys is None:
        namelistKeys = namelistDict.keys()
    namelistStr = namelistName + endl
    for key in namelistKeys:
        namelistStr += stl + key + eq + namelistDict[key] + endl
    namelistStr += '/' + endl
    return namelistStr

# Input file generator function

In [8]:
def SG2ibrav(spaceGroupIdx,SGTable):
    ibrav = None
    sgInfo = SGTable[SGTable['Table No.'] == spaceGroupIdx]
    bravaisLattice = sgInfo['Crystal System'].values[0].encode('utf-8') + ' ' + sgInfo['Full notation'].values[0][0].encode('utf-8')
    bravaisLattice2ibrav = {'cubic P':1,'cubic F':2,'cubic I':3,'hexagonal P':4,'trigonal P':4,'trigonal R':5,'tetragonal P':6,'tetragonal I':7,
                            'orthorhombic P':8,'orthorhombic A':9,'orthorhombic B':9,'orthorhombic C':9,'orthorhombic F':10,'orthorhombic I':11,
                            'monoclinic P':12,'monoclinic A':13,'monoclinic B':13,'monoclinic C':13, 'triclinic P':14}
    ibrav = bravaisLattice2ibrav[bravaisLattice]
   
    return ibrav

def SG2wyckoff(spaceGroupIdx,wyckTable):
    highestMultiplicitySite = wyckTable[spaceGroupIdx].ix[0,:]
    return str(highestMultiplicitySite['Multiplicity']) + highestMultiplicitySite['Wyckoff letter']

def SG2Postion(spaceGroupIdx,crystals):
    generalPosition = crystals[spaceGroupIdx][0].get_scaled_positions()[0]
    return generalPosition

def SG2cellParam(spaceGroupIdx,crystals):
    cellParam = crystals[spaceGroupIdx][0].get_cell_lengths_and_angles()
    return cellParam

def makeQEInnput(crystals,WyckTable,SGTable,ElemTable,
                 zatom = 14,rhocutoff = 20 * 4,wfccutoff = 20,kpt = [2,2,2],kpt_offset = [0,0,0]):
    inputDic = {}
    
    elemInfo = ElemTable[ElemTable['z'] == zatom]
    species = [elemInfo['sym'].values[0],]*1
    mass = [elemInfo['mass'].values[0]] # in atomic unit
    PP = ['Si.pbe-n-rrkjus_psl.1.0.0.UPF']
    nvalence = elemInfo['val'].values[0]
    
    for SpaceGroupIdx in crystals:
        wyck = [SG2wyckoff(SpaceGroupIdx,WyckTable)]
        positions = [SG2Postion(SpaceGroupIdx,crystals)]
        cellParam = SG2cellParam(SpaceGroupIdx,crystals)
        ibrav = SG2ibrav(SpaceGroupIdx,SGTable)
        multipicity = WyckTable[SpaceGroupIdx]['Multiplicity'][0]

        if int(0.2*multipicity*nvalence/2.) > 4:
            nbnd = int(multipicity*nvalence/2.+int(0.2*multipicity*nvalence/2.))
        else:
            nbnd = int(multipicity*nvalence/2.+4)
        
        # define name list of QE
        control = {'calculation' : '"scf"',
                  'outdir' : '"./out/"',
                  'prefix' : '"qe"',
                  'pseudo_dir' : '"./pseudo"',
                  'restart_mode' : '"from_scratch"',
                  'verbosity' : '"high"',
                  'wf_collect' : '.false.',
            }
        controlKeys = ['calculation','outdir','prefix','pseudo_dir', 'restart_mode','verbosity' ,'wf_collect']
        system = {
                  'ecutrho' :   '{:.5f}'.format(rhocutoff),
                  'ecutwfc' :   '{:.5f}'.format(wfccutoff),
                  'ibrav' : str(ibrav),
                  'nat' : str(1),
                  'nbnd' : str(nbnd),
                  'ntyp' : str(1),
                  'occupations' : '"smearing"',
                  'smearing' : '"cold"',
                  'degauss' :   '1.0000000000d-02',
                  'space_group' : str(SpaceGroupIdx),
                  'A' : str(cellParam[0]),
                  'B' : str(cellParam[1]),
                  'C' : str(cellParam[2]),
                  'cosAB' : '{:.5f}'.format(np.cos(cellParam[5]*np.pi/180.)),
                  'cosAC' : '{:.5f}'.format(np.cos(cellParam[4]*np.pi/180.)),
                  'cosBC' : '{:.5f}'.format(np.cos(cellParam[3]*np.pi/180.)),
            }
        syskeys = [ 'ecutrho','ecutwfc','ibrav', 'nat','nbnd','ntyp' ,
                   'occupations', 'smearing','degauss','space_group','A','B' ,'C', 'cosAB', 'cosAC', 'cosBC' ]
        electrons = {'conv_thr':'1.0000000000d-06'}

        # define cards of QE
        optionkey = 'unit'
        atomic_sp = {'ATOMIC_SPECIES':{'species':np.unique(species), 'mass':mass,'PP':PP},'unit':''}
        atspkeys = ['species','mass','PP']
        atomic_pos = {'ATOMIC_POSITIONS':{'species':species, 'wickoffs':wyck,'positions':positions},'unit':'crystal_sg'}
        atposkeys = ['species', 'wickoffs','positions']
        kpoints = {'K_POINTS':{'kpoints':kpt+kpt_offset},'unit':'automatic'}
        kptkeys = ['kpoints']
        
        qeInput = ''

        qeInput += makeNamelist('&CONTROL',control)
        qeInput += makeNamelist( '&SYSTEM' ,system,syskeys)
        qeInput += makeNamelist( '&ELECTRONS',electrons)
        qeInput += makeCard(atomic_sp,cardKeys = atspkeys,optionKey = optionkey,T = True) 
        qeInput += makeCard(atomic_pos,cardKeys = atposkeys,optionKey = optionkey,T = True) 
        qeInput += makeCard(kpoints,cardKeys = kptkeys,optionKey = optionkey,T = False)
        
        inputDic[SpaceGroupIdx] = qeInput
        
    return inputDic

## Load general information

In [13]:
fileNames = {}
dataDir = '../'
fileNames['crystals'] = 'Si-crystals-full-relax.pck'
fileNames['wyck'] = dataDir+'SpaceGroup-multiplicity-wickoff-info.pck'
fileNames['general info'] = dataDir+'SpaceGroup-general-info.pck'
fileNames['elements info'] = dataDir+'General-Info-Elements-fast.pck'

In [14]:
# give the SG 
spaceGroupIdxRange  = range(1,230+1)

Gen  = GenCrystalStructures(spaceGroupIdxRange=spaceGroupIdxRange)
Gen.setCrystals(fileNames['crystals'])
with open(fileNames['wyck'],'rb') as f:
    WyckTable = pck.load(f)
SGTable = pd.read_pickle(fileNames['general info'])
ElemTable = pd.read_pickle(fileNames['elements info'])

In [15]:
crystals = Gen.crystals

In [18]:
pck.dump?

In [20]:
with open('230spg-crystals.pck','wb') as f:
    pck.dump(crystals,f)

In [6]:
inputDic = makeQEInnput(Gen.crystals,WyckTable,SGTable,ElemTable)

In [7]:
print inputDic[50]

&CONTROL 
  wf_collect = .false. 
  calculation = "scf" 
  verbosity = "high" 
  pseudo_dir = "./pseudo" 
  prefix = "qe" 
  restart_mode = "from_scratch" 
  outdir = "./out/" 
/ 
&SYSTEM 
  ecutrho = 80.00000 
  ecutwfc = 20.00000 
  ibrav = 8 
  nat = 1 
  nbnd = 20 
  ntyp = 1 
  occupations = "smearing" 
  smearing = "cold" 
  degauss = 1.0000000000d-02 
  space_group = 50 
  A = 3.40012493343 
  B = 5.05457258726 
  C = 9.02605614869 
  cosAB = 0.00000 
  cosAC = -0.00000 
  cosBC = 0.00000 
/ 
&ELECTRONS 
  conv_thr = 1.0000000000d-06 
/ 
ATOMIC_SPECIES   
  Si  28.0855  Si.pbe-n-rrkjus_psl.1.0.0.UPF 
ATOMIC_POSITIONS  crystal_sg 
  Si  8m   0.75000817  0.87506799  0.59441631 
K_POINTS  automatic 
  2  2  2  0  0  0 



In [6]:
def make_dir(file_path):
    directory = os.path.abspath(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [7]:
#'/Users/iMac/Documents/Workdir/QMAT/check-QE-crystal_sg'
dirNames = {sg:'./data/sg-test_'+str(sg) for sg in range(1,230+1)}

In [11]:
import subprocess as sp
os.environ['TMPDIR'] = "/tmp/"

In [77]:
exitState = {}
for sg in dirNames:
    make_dir(dirNames[sg])
    inputName = os.path.abspath(dirNames[sg]+'/qe.in')
    outputName = os.path.abspath(dirNames[sg]+'/qe.out')
    errName = os.path.abspath(dirNames[sg]+'/qe.err')
    with open(inputName,'w') as f:
        f.write(inputDic[sg])
    param = '-in '+ inputName
    print param
    # Set up the echo command and direct the output to a pipe
    exitState[sg] = sp.call('mpirun -np 2 pw.x ' + param , stdout=open(outputName, 'w'), 
                             stderr=open(errName, 'w'), shell=True)
    if exitState[sg]:
        print sg


-in /Users/iMac/Dropbox/Felix_work/qmat/qe-sg-detection-test/sg_1/qe.in
1
-in /Users/iMac/Dropbox/Felix_work/qmat/qe-sg-detection-test/sg_2/qe.in
-in /Users/iMac/Dropbox/Felix_work/qmat/qe-sg-detection-test/sg_3/qe.in
-in /Users/iMac/Dropbox/Felix_work/qmat/qe-sg-detection-test/sg_4/qe.in
4
-in /Users/iMac/Dropbox/Felix_work/qmat/qe-sg-detection-test/sg_5/qe.in
-in /Users/iMac/Dropbox/Felix_work/qmat/qe-sg-detection-test/sg_6/qe.in
-in /Users/iMac/Dropbox/Felix_work/qmat/qe-sg-detection-test/sg_7/qe.in
7
-in /Users/iMac/Dropbox/Felix_work/qmat/qe-sg-detection-test/sg_8/qe.in
-in /Users/iMac/Dropbox/Felix_work/qmat/qe-sg-detection-test/sg_9/qe.in
9
-in /Users/iMac/Dropbox/Felix_work/qmat/qe-sg-detection-test/sg_10/qe.in
-in /Users/iMac/Dropbox/Felix_work/qmat/qe-sg-detection-test/sg_11/qe.in
-in /Users/iMac/Dropbox/Felix_work/qmat/qe-sg-detection-test/sg_12/qe.in
-in /Users/iMac/Dropbox/Felix_work/qmat/qe-sg-detection-test/sg_13/qe.in
-in /Users/iMac/Dropbox/Felix_work/qmat/qe-sg-detect

In [82]:
prbSg = []
for sg in exitState:
    if exitState[sg]:
        prbSg.append(sg)

In [83]:
print prbSg

[1, 4, 7, 9, 19, 29, 33, 38, 39, 40, 41, 76, 78, 138, 144, 145, 146, 148, 155, 160, 161, 166, 167, 169, 170, 182, 225, 226, 227, 228]
